# 概要
Pandasのtipsを見つけ次第まとめる

In [1]:
import numpy as np
import pandas as pd

## loc, ilocの確認
※ ixを使うのはもうやめよう
- locは列ラベルまたは行ラベルを指定する
- ilocは列番号または行番号を指定する
- iloc[A:B,A:B]のとき、A <= x < Bの番号が取得されることに注意する

In [10]:
# テストデータ
df_number = pd.DataFrame(np.arange(16).reshape(4,4), columns = ['A0', 'A1', 'A2', 'A3'])
df_number

,A0,A1,A2,A3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15


In [16]:
# locは列ラベルまたは行ラベルを指定する
# df_number.loc[1,1] 列に"1"というラベルはないのでエラーとなる
df_number.loc[1, 'A1']

5

In [18]:
df_number.loc[:, 'A1' : 'A3'] # 連続していれば「:」でまたいで指定可能

,A1,A2,A3
0,1,2,3
1,5,6,7
2,9,10,11
3,13,14,15


In [19]:
# ilocは列番号または行番号を指定する
df_number.iloc[1, 1]

5

In [23]:
df_number.iloc[1:3, 1:4] # iloc[A:B,A:B]のとき、A <= x < Bの番号が取得されることに注意する

,A1,A2,A3
1,5,6,7
2,9,10,11


In [2]:
## 階層構造のアクセス

In [8]:
# 階層構造のテストデータ
df_multi = pd.DataFrame(np.arange(16).reshape(4,4),
                        index=[['A','A','B','B'], [1,2,3,4]],
                        columns=[['C','C','D','E'], ['c1','c2','c1','c4']])
df_multi.index.names = ['INDEX1','INDEX2']
df_multi.columns.names = ['COL1','COL2']
df_multi

COL1            C       D   E
COL2           c1  c2  c1  c4
INDEX1 INDEX2                
A      1        0   1   2   3
       2        4   5   6   7
B      3        8   9  10  11
       4       12  13  14  15

In [7]:
# INDEX1='A'、INDEX2=1のデータにアクセスする
df_multi.loc['A',:].loc[1,:]

COL1  COL2
C     c1      0
      c2      1
D     c3      2
E     c4      3
Name: 1, dtype: int64

In [15]:
# COL1='C'、COL2=c1のデータにアクセスする
df_multi.loc[:,'C'].loc[:,'c1']

INDEX1  INDEX2
A       1          0
        2          4
B       3          8
        4         12
Name: c1, dtype: int64

## CSVファイルの読み込み
pandasとdaskの読み込み方法
daskの方が大容量データの読み込みに向いている  
テストデータのirisは「/home/jovyan/work/01_pandas_cheetsheet/data/iris.csv」にあるとする

In [37]:
# ディレクトリの確認
import os
cwd = os.getcwd()
print(cwd)
print(os.listdir(cwd+'/data'))
iris_path = cwd+'/data/iris.csv'

/home/jovyan/work/01_pandas_cheetsheet
['iris.csv']


In [41]:
# pandasでの読み込み
df_iris = pd.read_csv(iris_path, sep=',', header=0) # ヘッダーを省略する場合はheader=Noneとする
df_iris.head()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [42]:
# daskでの読み込み
import dask.dataframe as dd
df_iris_dask = dd.read_csv(iris_path).compute()
df_iris_dask.head()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


## データの横持ちから縦持ち変換 
long型（積み上げ型、縦型、縦持ち）とwide型（横型、横持ち）と呼ぶらしい  
横持ちにするのをpivot、縦持ちにするのををunpivotと呼ぶらしい  
縦持ちにしたい軸をindexにセットしてからstack関数で変換する。  
または、melt関数で縦持ちにもできる。というかmelt関数で良い。

In [47]:
# テストデータの作成
df_pivot = pd.DataFrame({'dim1': ['A', 'B', 'C'],
                         'x1': pd.Series([10, 20, 30], dtype='int64'),
                         'x2': pd.Series([40, 50, 60], dtype='int64')})
df_pivot

,dim1,x1,x2
0,A,10,40
1,B,20,50
2,C,30,60


In [60]:
# 横持ちから縦持ちへの変換
df_unpivot = df_pivot.set_index('dim1').stack().reset_index() # 縦持ちにしたい軸をindexにセットしてからstackする
df_unpivot.columns = ['dim1', 'x_dim', 'x_mes']
df_unpivot

,dim1,x_dim,x_mes
0,A,x1,10
1,A,x2,40
2,B,x1,20
3,B,x2,50
4,C,x1,30
5,C,x2,60


In [55]:
# melt関数を使った場合
pd.melt(df_pivot, id_vars=['dim1'], value_vars=['x1','x2'])

,dim1,variable,value
0,A,x1,10
1,B,x1,20
2,C,x1,30
3,A,x2,40
4,B,x2,50
5,C,x2,60


## データの縦持ちから横持ち
pivot関数を使う

In [61]:
df_unpivot.pivot(index = 'dim1', columns = 'x_dim', values = 'x_mes').reset_index()

x_dim,dim1,x1,x2
0,A,10,40
1,B,20,50
2,C,30,60
